In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import tensorflow_hub as hub

In [ ]:
# Cleaning the data 

def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [ ]:
# Reading the input file 
df = pd.read_csv("../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv")

# Cleaning the text from the dataframe
df['comment_text'] = df['comment_text'].map(lambda com : clean_text(com))   

# Taking out the values from the file as Train labels and examples 
train_labels = df['severe_toxicity'].values
train_examples = df['comment_text'].values

In [ ]:
# Reading the Test file for scoring purpose
df_test = pd.read_csv('../input/jigsaw-toxic-severity-rating/comments_to_score.csv')

# Cleaning out the test file text data
df_test['text'] =df_test['text'].map(lambda com : clean_text(com))

# Taking out the text values in the form of an ARRAY
testing_text = df_test['text'].values

In [ ]:
# Taking the Pretrained model from the Tensorflow hub
model = "../input/my-model-used-for-jigsaw" # <- Model name

# Taking out input shape for a model with a data type of string from a tensorflow with traninable model
hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)

In [ ]:
# Adding the final classification
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(50, activation = 'relu'))
model.add(tf.keras.layers.Dense(60, activation = 'relu'))
model.add(tf.keras.layers.Dense(50, activation = 'relu'))
model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))

model.summary()

In [ ]:
model.compile(optimizer = 'adam',
             loss = tf.keras.losses.BinaryCrossentropy(from_logits = False),
             metrics = ['accuracy'])

In [ ]:
history = model.fit(train_examples,
                    train_labels,
                    epochs = 1,
                    verbose = 1)

In [ ]:
predictions = model.predict(testing_text)
print(predictions)

In [ ]:
df_sample_file = pd.read_csv('../input/jigsaw-toxic-severity-rating/sample_submission.csv')
df_sample_file.head()

In [ ]:
df_sample_file['score'] = predictions
df_sample_file.to_csv('./submission.csv', index = False)